# Digital Barrier Option Pricing with Monte Carlo Simulation

This notebook explores the valuation of digital barrier options using Monte Carlo methods. Barrier options are a class of path-dependent derivatives whose payoff depends on whether the underlying asset price breaches a predetermined barrier level during the life of the option.

Unlike standard European options, digital barrier options provide a fixed payout (typically binary) if the barrier condition is met or not met, depending on the option type.

## Objectives

- Implement a Monte Carlo pricing model for digital barrier options.
- Analyze and compare different types of barrier options:
  - Up-and-In Call
  - Up-and-Out Call
  - Down-and-In Put
  - Down-and-Out Put
- Simulate the underlying asset price paths.
- Visualize the impact of barrier levels and volatility on option prices.
- Validate the model with sample input data.

## Assumptions

- Constant interest rate and volatility.
- Geometric Brownian Motion for asset price evolution.
- No dividends.
- Discrete monitoring of the barrier at each time step.



In [2]:
import numpy as np

def monte_carlo_digital_barrier(
    S: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    barrier: float,
    option_type: str = "call",
    barrier_type: str = "up-and-in",
    payout: float = 1.0,
    n_simulations: int = 10000,
    n_steps: int = 100
) -> float:
    

    # Validations
    if option_type not in {"call", "put"}:
        raise ValueError("option_type must be 'call' or 'put'")
    if barrier_type not in {"up-and-in", "up-and-out", "down-and-in", "down-and-out"}:
        raise ValueError("barrier_type must be one of 'up-and-in', 'up-and-out', 'down-and-in', 'down-and-out'")

    # Simulation parameters
    dt = T / n_steps
    discount_factor = np.exp(-r * T)
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt)

    # Generate price paths
    Z = np.random.normal(size=(n_simulations, n_steps))
    log_returns = drift + diffusion * Z
    log_paths = np.cumsum(log_returns, axis=1)
    price_paths = S * np.exp(log_paths)
    price_paths = np.concatenate([np.full((n_simulations, 1), S), price_paths], axis=1)

    # Check barrier condition
    barrier_hit = None
    if "up" in barrier_type:
        barrier_hit = np.any(price_paths >= barrier, axis=1)
    elif "down" in barrier_type:
        barrier_hit = np.any(price_paths <= barrier, axis=1)

    # Determine which paths result in payout
    if "in" in barrier_type:
        active_paths = barrier_hit
    else:  # "out"
        active_paths = ~barrier_hit

    # Payoff depends only on final spot relative to strike
    if option_type == "call":
        in_the_money = price_paths[:, -1] > K
    else:
        in_the_money = price_paths[:, -1] < K

    # Final payoff (digital fixed amount if both conditions met)
    payoffs = payout * (active_paths & in_the_money)

    return discount_factor * np.mean(payoffs)

#### up-and-in call

In [3]:
# Parameters
S = 195.00          # Current price of AAPL
K = 200.00          # Strike price
barrier = 205.00    # Barrier level
T = 0.25            # Time to maturity (in years)
r = 0.05            # Risk-free rate
sigma = 0.25        # Volatility
option_type = "call"
barrier_type = "up-and-in"
payout = 1.0

# Monte Carlo config
n_simulations = 10000
n_steps = 100

# Compute digital barrier price
price = monte_carlo_digital_barrier(
    S, K, T, r, sigma,
    barrier=barrier,
    option_type=option_type,
    barrier_type=barrier_type,
    payout=payout,
    n_simulations=n_simulations,
    n_steps=n_steps
)

print(f"{barrier_type.capitalize()} {option_type.capitalize()} on AAPL: ${price:.4f}")

Up-and-in Call on AAPL: $0.4145


#### Visualization

In [5]:
import plotly.graph_objects as go

# Rango de strikes y barreras
strike_range = np.linspace(180, 210, 20)
barrier_range = np.linspace(190, 220, 20)

Z = []
for B in barrier_range:
    row = []
    for K_val in strike_range:
        price = monte_carlo_digital_barrier(
            S=S,
            K=K_val,
            T=T,
            r=r,
            sigma=sigma,
            barrier=B,
            option_type=option_type,
            barrier_type=barrier_type,
            payout=payout,
            n_simulations=3000,  # reducimos para rapidez en la visualización
            n_steps=50
        )
        row.append(price)
    Z.append(row)

# Convertimos a superficie 3D
fig = go.Figure(data=[go.Surface(
    z=Z,
    x=strike_range,
    y=barrier_range,
    colorscale='Viridis'
)])

fig.update_layout(
    title='Digital Barrier Option Price',
    scene=dict(
        xaxis_title='Strike (K)',
        yaxis_title='Barrier (B)',
        zaxis_title='Option Price'
    ),
    autosize=True,
    height=700
)

fig.show()
